In [31]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "../data/train/"
categories = [name.split('_')[1] for name in os.listdir(caltech_dir)]
nb_classes = len(categories)

image_w = 32
image_h = 32

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(os.listdir(caltech_dir)):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img) / 255.0

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("../numpy_data/multi_image_data.npy", xy)

print("ok", len(y))

:  ../data/train//23_Jung Eunji\tri_0_3433.jpg
24_Jung Guk  파일 길이 :  2489
24_Jung Guk  :  ../data/train//24_Jung Guk\10.jpg
24_Jung Guk  :  ../data/train//24_Jung Guk\tri_0_1693.jpg
24_Jung Guk  :  ../data/train//24_Jung Guk\tri_0_2561.jpg
24_Jung Guk  :  ../data/train//24_Jung Guk\tri_0_339.jpg
25_Jung Haein  파일 길이 :  2489
25_Jung Haein  :  ../data/train//25_Jung Haein\10.jpg
25_Jung Haein  :  ../data/train//25_Jung Haein\tri_0_1704.jpg
25_Jung Haein  :  ../data/train//25_Jung Haein\tri_0_2565.jpg
25_Jung Haein  :  ../data/train//25_Jung Haein\tri_0_3399.jpg
26_Jung Hyungdon  파일 길이 :  2493
26_Jung Hyungdon  :  ../data/train//26_Jung Hyungdon\104.jpg
26_Jung Hyungdon  :  ../data/train//26_Jung Hyungdon\tri_0_1757.jpg
26_Jung Hyungdon  :  ../data/train//26_Jung Hyungdon\tri_0_2611.jpg
26_Jung Hyungdon  :  ../data/train//26_Jung Hyungdon\tri_0_3444.jpg
27_Jung Jihoon  파일 길이 :  2497
27_Jung Jihoon  :  ../data/train//27_Jung Jihoon\10.jpg
27_Jung Jihoon  :  ../data/train//27_Jung Jihoon\tr

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [6]:
X_train.shape

(145800, 32, 32, 3)

In [32]:
import pickle

xy = (X_train, X_test, y_train, y_test)
with open("../numpy_data/multi_image_data.npy",'wb') as f:
    pickle.dump(xy,f,pickle.HIGHEST_PROTOCOL)

# np.save("./numpy_data/multi_image_data.npy",)

In [13]:
import os
direc = '../data/train/'

for c in os.listdir(direc):
    for idx,f in enumerate(os.listdir(direc+c)):
        if idx<2400:
            continue
        os.remove(direc+c+'/'+f)


In [ ]:
import os
direc = '../data/validation/'

for c in os.listdir(direc):
    for idx,f in enumerate(os.listdir(direc+c)):
        if idx<400:
            continue
        os.remove(direc+c+'/'+f)


In [1]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend.tensorflow_backend as K

X_train, X_test, y_train, y_test = np.load('../numpy_data/multi_image_data.npy', allow_pickle=True)
print(X_train.shape)
print(X_train[0].shape)

Using TensorFlow backend.
(145800, 32, 32, 3)
(32, 32, 3)


In [9]:
import numpy as np
X_train, X_test, y_train, y_test = np.load('../numpy_data/multi_image_data.npy', allow_pickle=True)

OSError: Failed to interpret file '../numpy_data/multi_image_data.npy' as a pickle

In [33]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "../data/train/"
categories = [name.split('_')[1] for name in os.listdir(caltech_dir)]
nb_classes = len(categories)

In [38]:
with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3,3), padding="same", activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    model_dir = '../model'

    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    model_path = model_dir + '/model2.h5'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=2, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [8]:
X_train.shape[1:]

(32, 32, 3)

In [39]:
with K.tf_ops.device('/device:GPU:0'):
    history = model.fit(X_train, y_train, batch_size=512, epochs=50, validation_data=(X_test, y_test),validation_split=0.2, callbacks=[checkpoint, early_stopping])

Train on 238728 samples, validate on 79576 samples
Epoch 1/50
238728/238728 [==============================] - 292s 1ms/step - loss: 3.9829 - accuracy: 0.1090 - val_loss: 3.0503 - val_accuracy: 0.2915

Epoch 00001: val_loss improved from inf to 3.05027, saving model to ../model/model2.h5
Epoch 2/50
238728/238728 [==============================] - 300s 1ms/step - loss: 2.7413 - accuracy: 0.3192 - val_loss: 2.0921 - val_accuracy: 0.4728

Epoch 00002: val_loss improved from 3.05027 to 2.09214, saving model to ../model/model2.h5
Epoch 3/50
238728/238728 [==============================] - 291s 1ms/step - loss: 2.1080 - accuracy: 0.4544 - val_loss: 1.6952 - val_accuracy: 0.5707

Epoch 00003: val_loss improved from 2.09214 to 1.69516, saving model to ../model/model2.h5
Epoch 4/50
238728/238728 [==============================] - 292s 1ms/step - loss: 1.7378 - accuracy: 0.5370 - val_loss: 1.4169 - val_accuracy: 0.6347

Epoch 00004: val_loss improved from 1.69516 to 1.41692, saving model to ../m

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = '../model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/model2.h5'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=2, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [14]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, "../model/")

In [11]:
!pip install tensorflowjs

  Using cached tensorflowjs-1.7.4.post1-py3-none-any.whl (57 kB)
  Using cached tensorflow_cpu-2.2.0-cp37-cp37m-win_amd64.whl (189.3 MB)


In [22]:
import os
import json

os.listdir('../data/train')
datas = {}
for idx, data in enumerate(os.listdir('../data/train')):
    datas[idx] = data

with open("classes.json",'w') as f:
    json.dump(datas,f)

In [23]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, "C:\\Users\\kkho5\\Desktop\\project\\SimilarCeleb\\js")

In [14]:
model = Sequential()
model.add(Conv2D(64, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = '../model'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/multi_img_classification.model'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [15]:
with K.tf_ops.device('/device:GPU:0'):
    history = model.fit(X_train, y_train, batch_size=256, epochs=50, validation_data=(X_test, y_test),validation_split=0.2, callbacks=[checkpoint, early_stopping])

NameError: name 'K' is not defined

In [35]:
import numpy as np
import os
# 랜덤시드 고정시키기
np.random.seed(5)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=10,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  fill_mode='nearest')

for idx in range(14,15):
    categori = f"../train/{idx}/"
    for catimg in os.listdir(categori):
        if len(os.listdir(categori)) > 7500:
            break
        img = load_img(categori+"/"+catimg)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        i = 0

        # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
        for batch in data_aug_gen.flow(x, batch_size=1, save_to_dir=categori, save_prefix='tri', save_format='jpg'):
            i += 1
            if i > 120: 
                break

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '../train/14/'

7512
7392
7508
7503
7509
7525
7538


In [25]:
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# 데이터셋 불러오기
data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=10,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.1,
                                  zoom_range=[0.9, 1.1],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')

In [26]:
import numpy as np
import os
# 랜덤시드 고정시키기
np.random.seed(5)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

root = "../data/train/"
for idx,name in enumerate(os.listdir(root)):
    imgs = root+name
    for image in os.listdir(imgs):
        if len(os.listdir(imgs)) > 2400:
            break
        img = load_img(imgs+'/'+image)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        i = 0

        # 이 for는 무한으로 반복되기 때문에 우리가 원하는 반복횟수를 지정하여, 지정된 반복횟수가 되면 빠져나오도록 해야합니다.
        for batch in data_aug_gen.flow(x, batch_size=1, save_to_dir=imgs, save_prefix='tri', save_format='jpg'):
            i += 1
            if i > 120: 
                break
        

In [30]:
root = "../data/train/"
for idx,name in enumerate(os.listdir(root)):
    if idx > 80:
        os.rename(root+name,root+f'{idx}_'+name)